### Clean Spark DataFrame Sentence

In [ ]:
#!pip install modin[ray]

In [ ]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 

In [ ]:
#import pandas as pd
import modin.pandas as pd
import numpy as np
import re
import nltk
from nltk import word_tokenize

In [ ]:
# def clean_text(text):
#     text = ".".join(text)
#     text_clean_brackets = text.strip("-").strip("*").strip("/").strip("+").strip("#").strip("$")
#     text_clean_data_brackets = re.sub('[\(\[\{].*?[\)\]\}]', '', text_clean_brackets)
#     text_clean_list = re.findall(r'\w+.', text_clean_data_brackets)
#     text_result = (" ".join(text_clean_list)).lower()
#     return text_result 

In [ ]:
df = pd.read_csv("Spark_Sentence_Tokenize1.csv")
df

In [ ]:
df.result.loc[10]

In [ ]:
df.columns

In [ ]:
# def clean_text(text):
#     text_clean_list = re.findall(r'\w+', text)
#     alpha_num = [w for w in text_clean_list if w.isalnum()]
#     text_result = (" ".join(alpha_num)).lower()
#     return text_result 

In [ ]:
# def clean_text(text):
#     clean_punc = re.sub(r'[^\w\s]','',text)
#     text_clean_list = re.findall(r'\w+', clean_punc)
#     #text_clean_list = word_tokenize(clean_punc)
#     #text_clean_list = re.findall(r'\w+', text, flags=re.UNICODE)
#     #alpha_num = [w for w in text_clean_list if w.isalpha()]
#     text_result = (" ".join(text_clean_list)).lower()
#     return text_result 

In [ ]:
def clean_text(text):
    clean_brackets = text.strip("[]")
    clean_brackets_data = re.sub('[\(\[\{].*?[\)\]\}]', '', clean_brackets)
    clean_punc = re.sub(r'[^\w\s]','',clean_brackets_data)
    text_clean_list = re.findall(r'\w+', clean_punc)
    #alpha_num = [w for w in text_clean_list if w.isalnum()]
    text_result = (" ".join(text_clean_list)).lower()
    return text_result 

In [ ]:
sent_list = []
for i in df.result:
    i = clean_text(i)
    sent_list.append(i)  

In [ ]:
df_clean = pd.DataFrame(sent_list)
df_clean.head()

In [ ]:
df_clean = df_clean.rename(columns={0:"sentence"})
df_clean

In [ ]:
df_clean.drop(df_clean[df_clean.sentence == ""].index, inplace=True)

In [ ]:
sent_count = df_clean.sentence.value_counts().sort_values(ascending=False)
#sent_count

In [ ]:
sent_count_df = pd.DataFrame(sent_count).reset_index()
sent_count_df.rename(columns={"index":"sentence", "sentence":"frequency"}, inplace=True)

In [ ]:
total_frequency = sent_count_df.frequency.sum()

In [ ]:
sent_count_df["ratio"] = (sent_count_df.frequency/total_frequency)*100

In [ ]:
sent_count_df["cumul_ratio"] = np.cumsum(sent_count_df["ratio"])

In [ ]:
# def sentence_lenght(sentence):
#     word_var2 = re.findall(r'\w+', sentence)
#     if len(word_var2) <= 10:
#         sent_var2 = " ".join(word_var2)
#         return sent_var2
#     else:
#         return "sentence is bigger than ten word"

In [ ]:
# # select less and equal ten word sentence
# def sentence_lenght(sentence):
#     word_var2 = word_tokenize(sentence)
#     if len(word_var2) <= 10:
#         sent_var2 = " ".join(word_var2)
#         return sent_var2
#     else:
#         return "sentence is bigger than ten word"

In [ ]:
def sentence_lenght(sentence):
    word_var2 = re.findall(r'\w+', sentence)
    if len(word_var2) <= 10:
        return sentence
    else:
        return "sentence is bigger than ten word"

In [ ]:
sent_count_df.sentence = sent_count_df.sentence.apply(sentence_lenght)

In [ ]:
sent_count_df.drop(sent_count_df[sent_count_df.sentence == "sentence is bigger than ten word"].index,axis=0, inplace=True)
sent_count_df.reset_index(drop=True, inplace=True)

In [ ]:
df_result = sent_count_df[sent_count_df.frequency>1]

In [ ]:
df_result.to_csv("Clean_Spark_Sentence_Tokenize1.csv", index=False)